# Grant Checker
### To-do:

#### - Do categorical matching (Focus on OOE and EQPT)
- Add loading page and show instructions on how to use 
- Improve categorical lists, allow user to add own words in selected lists
- Warn to only open excel when the program shows 'Done'

- If upload file is excel..., if pdf..., if word doc...

- Improve word capturig, test with actual grant pdf/excel

- Edit date function

# Imports

In [1]:
# =========================================================================
# =============================== IMPORTS ===============================
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import PIL.Image
from time import sleep 

import xlrd
import csv

import numpy as np
from numpy import column_stack, arange, pi, sin

import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

from gensim.models import Word2Vec, Doc2Vec, KeyedVectors

import smart_open
from smart_open import open

import nltk
# nltk.download()
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import StanfordNERTagger

import re, datetime

from dateutil.parser import parse
from monthDictionary import monthDictionary

import pycountry

import requests

# ============= PDF ===============
# Import libraries 
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
from PyPDF2 import PdfFileReader

# import pyocr
# import pyocr.builders
# =================================

import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
from openpyxl.cell import Cell

from openpyxl import Workbook

import os
import glob,os
import json
from tqdm import tqdm

# ============= LISTS ===============
from eqpt_approved_list import eqpt_approved_list
from eqpt_disapproved_list import eqpt_disapproved_list
from ooe_approved_list import ooe_approved_list
from ooe_disapproved_list import ooe_disapproved_list
from overseas_disapproved_list import overseas_disapproved_list
# ===================================


# ============= CATEGORICAL LISTS ===============
from it_eqpt import it_eqpt 
# if the grant item description contains the word "it equipment/computer/laptop/software..." 
# the excel item description will

# apparatus and chemicals etc.
from lab_eqpt import lab_eqpt

# ===============================================


# =========================== Set environmental variables programmatically ==============================
#Set the classpath to the path where the jar file is located
# os.environ['CLASSPATH'] = r"C:\Users\Nicole\Documents\ASTAR Projects\Grant Project\stanford-ner-2018-10-16/stanford-ner.jar"

# MAC
os.environ['CLASSPATH'] = "/Users/nicoleong/Documents/Anaconda/resources/stanford-ner-2018-10-16/stanford-ner.jar"

# ===============================================
#Set the Stanford models to the path where the models are stored
# os.environ['STANFORD_MODELS'] = r'C:\Users\Nicole\Documents\ASTAR Projects\Grant Project\stanford-english-corenlp-2018-10-05-models'

# MAC
os.environ['STANFORD_MODELS'] = '/Users/nicoleong/Documents/Anaconda/resources/stanford-english-corenlp-2018-10-05-models'

# ===============================================
#Set the java jdk path
# java_path = r"C:\Program Files\Java\jdk1.8.0_131\bin/java.exe"
# WINDOWS

# MAC
java_path = "/Users/nicoleong/Documents/Anaconda/resources/Java/jdk1.8.0_131/bin/java"
os.environ['JAVAHOME'] = java_path

# ===============================================
#Set the path to the model that you would like to use
# stanford_classifier = r"C:\Users\Nicole\Documents\ASTAR Projects\Grant Project\stanford-ner-2018-10-16\classifiers\english.all.3class.distsim.crf.ser.gz"
stanford_classifier = "/Users/nicoleong/Documents/Anaconda/resources/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz"
# =============================================================================================================


# ======================= Prepare countries array for use later in check_venue method =======================
countries = []

for country in list(pycountry.countries):
    countries.append(country.name)
   
countries.remove('Singapore')
# =============================================================================================================


# Model

In [2]:
# =====================================================================
# =============================== MODEL ===============================
# crawl model 
model = KeyedVectors.load_word2vec_format('../crawl-300d-2M.bin', binary = True)


# Methods to get Excel data
### I need to get the Project ID, project staff/manpower list, and dates in the grant form to compare to:
- Get the relevant rows in the same Project ID 
- See if the name (if stated) in the item description of the excel is in the project staff list
- See if the dates mentioned in the excel agree with the grant form dates

In [3]:
# ===================================================================================
# =============================== READ ORIGINAL EXCEL ===============================
# Things that need to be directly compared between the excel and grant proposal form: 

# ----- Project ID to get just the relevant rows 
# ----- Project staff / manpower list 
# ----- Dates of purchase/ travel needs to fall within correct period 

# Item description in excel sheet to grant proposal item description 
# (need a json file of accepted words - e.g. ooe approved list)


# ================= Get relevant df of matching project ID =================
def get_rel_excel_df(og_excel_df, grant_id):
    relevant_excel_df = og_excel_df.loc[og_excel_df['ProjectIDNumber'].str.contains(grant_id)]
    return relevant_excel_df



# ================= Get Fund Item ID row =================
def get_fund_item_row(og_excel_df, excel_fundItemID):
    row = og_excel_df.loc[og_excel_df['FundReqItemID'].str.contains(excel_fundItemID)]
    return row


# ================= Get name from relevant excel rows item description =================
# get the PERSON tag from the excel item description - typically this would be a name
# then check if this name is in the grant proposal project staff / manpower list (split by ',' to get these names)
# if not then highlight/flag 

# works on one row at a time, so need to loop this

# made it get more than one name just in case the description has a few PERSON tags
def get_excel_name(rel_excel_item_description):    
    #Build NER tagger object
    st = StanfordNERTagger(stanford_classifier)
    tagged = st.tag(str(rel_excel_item_description).split())

    names = [] 
    
    for tag in range(len(tagged)):
        if tagged[tag][1] == 'PERSON':
            print(tagged[tag][0])
            names.append(tagged[tag][0])
            
        else: 
            pass
    
    return names



# ================= Get the date(s) from the excel item description =================  
# The date is normally mentioned in the item description 
# date parse isn't really accurate right now - depends on how the person input the date
def get_excel_dates(rel_excel_votecode, rel_excel_item_description):
    # overseas always needs to have a date stated
    if "OVERSEAS" in rel_excel_votecode:
        # get the dates from the description
        
        # date parse isn't really accurate right now - depends on how the person input the date
        months = list(monthDictionary.keys())
    
        for month in months:
            if month in description:
                rel_excel_item_description = rel_excel_item_description.replace(month, month_format(month))

                
        # checking if a date was mentioned in the description        
        try:
            # remove letters from the description so only numbers and other characters are left
            rel_excel_item_description = re.sub(r'[a-zA-Z]', '', rel_excel_item_description)
            
            result = parse(rel_excel_item_description, fuzzy_with_tokens=True)
#             print(result[0]) #the date
            
            # can continue to do something with the date
            
            # if it's too near the posting date?
            # if the person already ends work before that date?
            #..
            
            date = result[0]
            
        except:
#             date = "Not fundable, the date of the overseas travel grant request were not stated"
            # if no date mentioned then let it pass
            date = ""

        
    # not OVERSEAS votecode
    else:
        try:
            rel_excel_item_description = re.sub(r'[a-zA-Z]', '', rel_excel_item_description)
            result = parse(rel_excel_item_description, fuzzy_with_tokens=True)
            date = result[0]
            
        except:
            # if no date mentioned then let it pass
            date = ""

    return date



# Methods to get grant pdf form data
### Also need he method to convert PDF to image and perform OCR 
### And the method to make a dataframe from the text extracted from the PDF grant form 
#### The method above requires methods that use regular expressions to get each relevant data field for the df

In [4]:
# ==============================================================================
# =============================== READ PDF GRANT ===============================
# 1. Method to convert PDF to image to allow OCR to capture text
# 2. Method to make captured text a dataframe to be used easily for the checking methods 


# Method to get the relevant text data from the excel sheet
# def proposal_excel_text(folder, file_name): 
#     for excel in glob.glob(os.path.join(folder, "*.xlsx")):
#         #...

# Method to convert PDF to an image
def pdf_to_image(folder, file_name):
    
    for pdf in glob.glob(os.path.join(folder, "*.pdf")):
        
        name = file_name.replace(".pdf", "")
        file_path = folder+'/'+file_name
        
        # Store all the pages of the PDF in a variable 
        pages = convert_from_path(file_path, 500) 

        # Counter to store images of each page of PDF to image 
        image_counter = 1

        # Iterate through all the pages stored above 
        for page in pages: 
            # PDF page n -> page_n.jpg 
            filename = "page_"+name+"_"+str(image_counter)+".jpg"
            page.save(filename, 'JPEG') 
            image_counter = image_counter + 1

        # Variable to get count of total number of pages 
        filelimit = image_counter-1

        # Creating a text file to write the output 
        outfile = "out_text_"+name+".txt"

        # Open the file in append mode so that  
        # All contents of all images are added to the same file 
        f = open(outfile, "w") 

        # ========================================
        # WINDOWS
#         pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
#         pytesseract.pytesseract.tesseract_cmd = r"../resources/Tesseract-OCR/tesseract.exe"

        # MAC 
        pytesseract.pytesseract.tesseract_cmd = "/usr/local/Cellar/tesseract/4.1.0/bin/tesseract"
        # ========================================
        
        # Iterate from 1 to total number of pages 
        for i in range(1, filelimit + 1): 

            filename = "page_"+name+"_"+str(i)+".jpg"
            text = str(((pytesseract.image_to_string(Image.open(filename).convert('L'))))) 
            text = text.replace('-\n', '')    

            # Finally, write the processed text to the file. 
            f.write(text) 
    #         print(text)

        # Close the file after writing all the text. 
        f.close() 

        # SHOW IN TKINTER
#         tk.Label(window, text = 'Text in grant proposal pdf form:' + text).pack()

        return text



# Method to make a dataframe out of the text in pdf_to_image
def make_df(text):
    grantId = get_pdf_id(text)
    name = get_name(text)
    projectTitle = get_project_title(text)
    staffList = get_project_staff(text)
    grantPeriod = get_grant_period(text)
    endDate = get_end_date(text)
    itemType = get_item_type(text)
    voteCode = get_vote_code(text)
    traveller = get_traveller(text)
    accountDescription = get_account_desc(text)
    itemDescription = get_item_desc(text)
    dateOfApplication = get_date_of_app(text)
    grantAmount = get_grant_amount(text)

    data = {'ProjectIDNumber': grantId,
            'Name': name, 
            'ProjectTitle': projectTitle, 
            'StaffList': staffList,
            'GrantPeriod': grantPeriod,
            'EndDate' : endDate,
            'ItemType': itemType,
            'VoteCode': voteCode,
            'Traveller': traveller,
            'AccountDescription': accountDescription,
            'ItemDescription': itemDescription,
            'DateOfApplication': dateOfApplication,
            'GrantAmount': grantAmount} 
    
    df = pd.DataFrame(data, index=[0])
    
    return df


# =======================================================================
# ======================= METHODS FOR TEXT IN PDF =======================

# This method formats strings into a list of just the important words from the string suitable to query from the specified model
def word_format_model(words, model):
    lemmatizer = WordNetLemmatizer()
    tokenized = [word.lower() for word in word_tokenize(words)]
    
    stop_words = stopwords.words('english')
    stop_words.extend(["'s", "&", "''", '""', '``', ',', '.', '!', '?', '[', ']', '(', ')', '{', '}', '<', '>', '/', '@', '#', '$', '%', '*', ':', ';', '-', '—', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    result = []
    
    for w in tokenized:
        if lemmatizer.lemmatize(w) not in stop_words:
            if lemmatizer.lemmatize(w) in model.vocab:
                result.append(lemmatizer.lemmatize(w))
              
    return result


# This method formats month from int to a string
def month_format(monthString):
    # Check monthDictionary
    monthNum = monthDictionary.get(monthString, "Not found")
                        
    return monthNum

# EXTRACT PROJECT ID
def get_pdf_id(text):
    valid = re.compile(r"[Ii][Dd][\:\s\w]+")
    pdf_id = valid.search(text)
    
    pdf_id = pdf_id.group()
    pdf_id = pdf_id.replace(": ", "")
    pdf_id = pdf_id.replace("id", "")
    pdf_id = pdf_id.replace("number", "")
    
    return pdf_id


# EXTRACT NAME
def get_name(text):
    valid = re.compile(r"[Nn]ame?[\:\s\w]+")
    name = valid.search(text)
    
    name = name.group()
    name = name.replace(":", "")
    name = name.replace("name", "")
    name = name.replace(" of applicant", "")
    
    return name


# EXTRACT PROJECT TITLE
def get_project_title(text):
    valid = re.compile(r"[Tt]itle?[\:\s\w]+")
    projectTitle = valid.search(text)
    projectTitle = projectTitle.group()

    projectTitle = projectTitle.replace(":", "")
    projectTitle = projectTitle.replace("title", "")
    
    return projectTitle


# EXTRACT PROJECT STAFF
def get_project_staff(text):
    valid = re.compile(r"[Ss]taff?[\:\s\w,]+")
    pdf_staff = valid.search(text)
    pdf_staff = pdf_staff.group()

    pdf_staff = pdf_staff.replace(":", "")
    pdf_staff = pdf_staff.replace("staff", "")
    
    return pdf_staff


# EXTRACT GRANT PERIOD (QUARTER AND YEAR)
# is this the period when the grant is valid? kind of short?
def get_grant_period(text):
    validQuarter = re.compile(r"[Qq]uarter?.+?\d")
    quarter = validQuarter.search(text)
    quarter = quarter.group()
    
    validYear = re.compile(r"[Yy]ear?.+?\d\d?\d?\d")
    year = validYear.search(text)
    year = year.group()
    
    grantPeriod = quarter + ' ' + year
    grantPeriod = grantPeriod.replace(":", "")
    grantPeriod = grantPeriod.replace(")", "")
    
    return grantPeriod


# EXTRACT GRANT END DATE
def get_end_date(text):
    valid = re.compile(r"[Ee]nd\s[Dd]ate?[\:\d\s\-\/]+")
    endDate = valid.search(text)
    endDate = endDate.group()

    endDate = endDate.replace(":", "")
    endDate = endDate.replace("end date", "")

    return endDate


# EXTRACT ITEM TYPE
def get_item_type(text):
    valid = re.compile(r"[Ii]tem\s[Tt]ype?[\:\s\w]+")
    itemType = valid.search(text)
    itemType = itemType.group()

    itemType = itemType.replace(":", "")
    itemType = itemType.replace("item type", "")
    
    return itemType


# EXTRACT VOTE CODE
def get_vote_code(text):
    valid = re.compile(r"[Vv]ote\s[Cc]ode?[\:\s\w\(\/]+")
    voteCode = valid.search(text)
    voteCode = voteCode.group()

    voteCode = voteCode.replace(":", "")
    voteCode = voteCode.replace("vote code", "")
    
    return voteCode


# EXTRACT TRAVELLER
def get_traveller(text):
    valid = re.compile(r"[Tt]raveller?[\:\s\w]+")
    traveller = valid.search(text)
    traveller = traveller.group()

    traveller = traveller.replace(":", "")
    traveller = traveller.replace("traveller", "")
    
    return traveller


# EXTRACT ACCOUNT DESCRIPTION
def get_account_desc(text):
    valid = re.compile(r"[Aa]ccount\s[Dd]escription?[\:\s\w]+")
    accountDescription = valid.search(text)
    accountDescription = accountDescription.group()

    accountDescription = accountDescription.replace(":", "")
    accountDescription = accountDescription.replace("account description", "")
    
    return accountDescription


# EXTRACT ITEM DESCRIPTION
def get_item_desc(text):
    valid = re.compile(r"[Ii]tem\s[Dd]escription?[\:\s\w\*\-\/]+")
    itemDescription = valid.search(text)
    itemDescription = itemDescription.group()

    itemDescription = itemDescription.replace(":", "")
    itemDescription = itemDescription.replace("item description", "")
    
    return itemDescription


# EXTRACT DATE OF APPLICATION
def get_date_of_app(text):
    valid = re.compile(r"[Dd]ate\sof\s[Aa]pplication?[\:\d\s\-\/]+")
    dateOfApplication = valid.search(text)
    dateOfApplication = dateOfApplication.group()

    dateOfApplication = dateOfApplication.replace(":", "")
    dateOfApplication = dateOfApplication.replace("date of application", "")

    return dateOfApplication


# EXTRACT GRANT AMOUNT 
def get_grant_amount(text):
    valid = re.compile(r"[$]\s?\d+?[.]\d+")
    grantAmount = valid.search(text)
    grantAmount = grantAmount.group()
    
    return grantAmount



# Methods to perform checking 
### Things that need to be compared/checked: 
- See if the name in the excel item description is in the Project staff list from the grant PDF form
- See if the date(s) in the excel item description is before the end date stated in the grant PDF form 
- See if the excel item description is similar to the grant pdf form excel description 

#### Other checks to be made for excel sheet data: 
- Check if have traveller if votecode is OVERSEAS
- Check if the item description item falls in an approved list of items or disapproved based on the votecode 
- Check if the cost is an exact cost or estimated cost
- if OVERSEAS votecode, check to see if the venue is out of Singapore with a web search

#### Return a score for each method and if the score is:
- 0.0-0.3 --> red highlight
- 0.3 to 0.7 --> yellow highlight 
- 0.7-1.0 --> pass, do nothing 


In [5]:
# =================================================================================
# =============================== COMPARING METHODS ===============================

# -- Not a staff under the project then flag
def staff_in_project(excel_name, project_staff_list):
    result = ""
    
    for name in excel_name: 
        if name.lower() in project_staff_list: # Approved, name is in the project staff list
            result = "-"
            break
        else:
#             print(excel_name + " not in project staff list")
            result = excel_name + " not in project staff list | "
    
    return result
 

# =====================================================================================================================

# -- DATE
def check_dates(excel_date, grant_end_date):
    if excel_date == "": # no date mentioned in the excel item description
        result = ""
    else: 
        result = "Need to edit date method. "
        # if the grant end date is after the date stated in the excel 
#         if grant_end_date > excel_date: # Fundable, date is within range
#             result = "-"
#         else: # if after 
#             result = "Date of purchase is after the end date for the grant | "
            
    return result


# =====================================================================================================================

# -- CHECK BY CATEGORY
# Have python lists of all words that would correspond to a certain category 
# loop through words in the grant/excel item description 
# if the excel/grant item description word in the list then replace the word with the umbrella term of the list 
# so that the similarity score will be higher 

# -- Find out if there is a 'match' between the Excel FundReqItemDescription 
# and Grant PDF item description --> make sure the similarity score is over a certain amount // 
# If item is not under the project then also flag 
def relevant(grant_description, excel_description, score):
        
    # First, check the grant description for a word to know which list to look into (e.g. it equipment --> then look at the it_eqpt list)
    
    # After that, loop through each word in the list and see if the excel item description contains a word in the list 
    # If it contains then replace the excel_description with "it equipment" (for example stated above)
 
    result1 = ""

    for word in lab_eqpt:
        if word in grant_description.lower():
            # After that, loop through each word in the list and see if the excel item description contains a word in the list 
            # If it contains then replace the excel_description with "it equipment" (for example stated above)

            for w in lab_eqpt:
                if w in excel_description.lower(): 
                    print("in lab_eqpt")
                    excel_description = word
                    result1 = "changed"

                else: 
                    result1 = "no"

        else:
            continue

    print(result1)
    print(excel_description)

    # if the result was "no" then check through the next list
    if result1 == "no":
        for word in it_eqpt:
            if word in grant_description.lower(): 
                for w in it_eqpt:
                    if w in excel_description.lower():
                        print("in it_eqpt")
                        excel_description = word
                        result1 = "changed"

                    else:
                        result1 = "no"

            else: 
                continue
    
    # With this, the relevance score will go up
    
    
    try:
        excel_description = word_format_model(excel_description, model)
        grant_description = word_format_model(grant_description, model)

        if excel_description == "":
            pass

        similarity = model.n_similarity(excel_description, grant_description)

        # threshold setting
        if float("{:.4f}".format(similarity)) >= float(score):
            result = "-"
        else:
            # Do more checks - he categorical checking
            # If the grant item description has a word in one of the categorical lists 
            # Then check if the excel item description also has something in that list
            
            result = "Excel item description NOT relevant to the grant proposal | "
            
        return result

    except ZeroDivisionError:
        result = "Algorithm could not understand the excel item description, please do manual check | "
        return result



# ================================================================================
# =============================== CHECKING METHODS ===============================

# -- If VoteCode is OVERSEAS, see if there is a traveller for the item --> if no then flag (0.0), if yes then...
# use the relevant excel df
def check_traveller(votecode, description):
    if votecode == "overseas":
        
        traveller = get_excel_name(description)
        print(traveller)
    
        if traveller != []: # true - Approved, a traveller was stated
            result = "-"
        else: # false - traveller not stated
            result = "Please indicate who is travelling | "
    else: # true - Not related to overseas travel, pass 
        result = "-"
        
    return result

# =====================================================================================================================


# -- Certain grant requests fall under the OOE VoteCode,
# some OVERSEAS, so make sure what they are requesting is under the correct VoteCode, otherwise flag
# and alert that need to reclass to whichever is the appropriate VoteCode

# There is an approved OOE/EQPT list, if item not inside that list then flag

# Maybe check it against an OOE list and if the item description includes the name of a country then 
# flag as it should be overseas if it isn't already
def check_votecode(df, votecode):
    
    # if it includes words found in the item list, it should be flagged and reclassified 
    # e.g. USA poster contains 'poster' and should not be classified as OVERSEAS
    # may need to tweak this with a similarity score threshold
    if "overseas" in votecode:
        if df.iloc[0]['FundReqItemDescription'] in (overseas_disapproved_list):
#             print("The item description was in the overseas disapproved list")
            result = "The item description was in the overseas disapproved list | "
        else:
            result = "-"

    #################################################################  
    elif "ooe" in votecode:
        
        if df.iloc[0]['FundReqItemDescription'] in (ooe_disapproved_list): 
#             print("The item description was in the ooe disapproved list")
            result = "The item description was in the ooe disapproved list | "
            return result
             
        # check if the description is in the approved list 
        if df.iloc[0]['FundReqItemDescription'] in (ooe_approved_list):
            result = "-"
            return result
        
        # if it's not then check in the other list
        if df.iloc[0]['FundReqItemDescription'] in (eqpt_approved_list):
#             print("The item description had an OOE votecode but was in the eqpt approved list")
            result = "The item description had an OOE votecode but was in the eqpt approved list | "
            return result

            # if it's in this list then flag and return the comment 
            # that the item should probably be reclassified to the other vote code 
        else: # if it's not there either then flag
            # str(df.iloc[0]['FundReqItemDescription']) + " had an OOE votecode but wasn't in any approved or disapproved list | "
            result = str(df.iloc[0]['FundReqItemDescription']) + " had an OOE votecode but wasn't in any approved or disapproved list "
            return result
        
        
    ################################################################# 
    elif "eqpt" in votecode:
        if df.iloc[0]['FundReqItemDescription'] in (eqpt_disapproved_list): 
#             print("The item description was in the eqpt disapproved list")
            result = "The item description was in the eqpt disapproved list | "
            return result
             
        if df.iloc[0]['FundReqItemDescription'] in (eqpt_approved_list):
            result = "-"
            return result
        
        if df.iloc[0]['FundReqItemDescription'] in (ooe_approved_list):
#             print("The item description had an EQPT votecode but was in the ooe approved list")
            result = "The item description had an EQPT votecode but was in the ooe approved list | "
            return result

        else: # if it's not there either then flag
            result = str(df.iloc[0]['FundReqItemDescription']) + " had an EQPT votecode but wasn't in any approved or disapproved list "
            return result
        
        
    #################################################################        
    else:
        print("Invalid vote code")
        result = "Invalid vote code"
        
    return result
    
    

# -- Actual cost not stated, then not fundable (accrual, accrued, estimated - not allowed)
def actual_cost(df):
#     reject_words = ['accrual', 'accrued', 'estimated']
    reject_words = ['estimated']
    
    # loop through the rejected words to see if the item description contains them - if not then all good
    for word in reject_words:
        if word in df.iloc[0]['FundReqItemDescription'].lower():
            result = "Estimated cost was stated, not fundable | "
        else:
            result = "-"
    
    return result
# =====================================================================================================================


# -- Not enough details/no description of items in description, flag
# if the description mentions an event - do a search and crawl for the information about that event. 
# If the event is some kind of concert etc., then no go



# =====================================================================================================================
# -- VENUE
# It'd be cool if the description mentions an event - do a search and crawl for the information 
# about that event. 
def check_venue(votecode, description):
    if "overseas" in votecode:

        result = ""
        
        countries.extend(['Beijing', 'SuZhou', 'San Franc', 'UK', 'US', 'USA', 'ShenZhen', 'GuangZhou', 'Xiamen'])
        
        # This one focuses on venue (make sure it's in an overseas country)
        # make sure that the conference/event is really valid (not local)
        for country in countries:
            if country.lower() in description.lower():
#                 print("Found a foreign country in the description")
                result = "-"
                  
 
        # get the name of the conference
        # do a search for that event 
        # get the venue/address of that event to make sure it's not in Singapore

        if result != "-":
            url = "http://www.google.com/search?q=" + description + "&btnI"
            r = requests.get(url, headers='')

            # do a feeling lucky search and see if the html contains 'Singapore'
            if 'Singapore' in r.text:
#                 print("This event is held in Singapore hence funding is not allowed under the OVERSEAS vote code")
                result = "This event is held in Singapore hence funding is not allowed under the OVERSEAS vote code | "

            else:
                # try to find an overseas country in the r.text
                for country in countries:
                    if country.lower() in r.text.lower():
#                         print("Found a foreign country in html") - Approved overseas country |
                        result = "-"
                        
                if result != "-":
#                     print("No overseas country was mentioned in this item description where the vote code was OVERSEAS, and the event details could not be found")
                    result = "No overseas country was mentioned in this item description where the vote code was OVERSEAS, and the event details could not be found | "
        
    else:
#         print("OOE / EQPT default approved")
        result = "-"
        
    # =========================================================
    # be specific on what item is being funded
    # e.g. not just online consumables, general purposes, Recl expns, research, Items are for research/work 
    # or if description doesn't contain any words (even if it's just numbers), flag
    # - need to name the item itself
    
    # kind of handled in check_votecode() because that method checks against the approved OOE and EQPT list
#     elif "OOE" in votecode:
        
        
#     elif "EQPT" in votecode:

#       else:
#         print("Invalid vote code")
#         result = False 
    # =========================================================
  
    return result



In [6]:
# ================================================================================
# =============================== HIGHLIGHT METHOD ===============================
def highlight(text, og_wb, score, check6Bool, check7Bool):    
    
    # ================================================================================================
    # =============================== Upload PDF and get text and data ===============================

#     text = pdf_to_image('test.pdf') # pdf filename will be uploaded by user

    # =============================================================================
    text = text.lower()
    text = text.replace("name", "| name")
    text = text.replace("project title", "| project title")
    text = text.replace("staff", "| staff")
    text = text.replace("grant date", "| grant date")
    text = text.replace("end date", "| end date")
    text = text.replace("item type", "| item type")
    text = text.replace("vote code", "| vote code")
    text = text.replace("traveller", "| traveller")
    text = text.replace("account description", "| account description")
    text = text.replace("item description", "| item description")
    text = text.replace("date of application", "| date of application")
    text = text.replace("grant amount", "| grant amount")
    print(text)

    # Make a dataframe from the text captured on the PDF Grant proposal form    
    # This will only be one row 
    grant_df = make_df(text)
    # from this grant_df, can get Project ID, staff list, item description, end date 
    grant_id = str(grant_df.iloc[0]['ProjectIDNumber']).strip()
    grant_staff = str(grant_df.iloc[0]['StaffList']).strip()
    grant_item_desc = str(grant_df.iloc[0]['ItemDescription']).strip()
    grant_end_date = str(grant_df.iloc[0]['EndDate']).strip()


    # =============================================================================
    # =============================== Upload Excel  ===============================

#     og_wb = openpyxl.load_workbook(og_wb_name) # excel will be uploaded by user

    # =============================================================================
    og_sheet = og_wb.active

    og_excel_df = pd.DataFrame(og_sheet.values)
    og_excel_df.columns = og_excel_df.iloc[0]
    
    # Convert all IDs in the df to string
    og_excel_df['ProjectIDNumber'] = og_excel_df['ProjectIDNumber'].apply(str)
    og_excel_df['FundReqItemID'] = og_excel_df['FundReqItemID'].apply(str)
    

    # from the excel df, get the relevant rows 
    rel_excel_df = get_rel_excel_df(og_excel_df, grant_id)

    # Make all text lowercase
    rel_excel_df = rel_excel_df.apply(lambda x: x.astype(str).str.lower())

    # from relevant rows get the vote code and item description which will inlcude date and name (will have to use these in a loop)
    rel_excel_votecode = rel_excel_df['VoteCode']
    rel_excel_item_description = rel_excel_df['FundReqItemDescription']
    rel_excel_name = get_excel_name(rel_excel_item_description)
    rel_excel_dates = get_excel_dates(rel_excel_votecode, rel_excel_item_description)
    # get_excel_dates(og_excel_df['VoteCode'][1], og_excel_df['FundReqItemDescription'][1])



    # handle highlighting from second row
    

    for row in range(len(rel_excel_df)):
        rel_excel_votecode = rel_excel_df.iloc[row]['VoteCode']
        rel_excel_item_description = rel_excel_df.iloc[row]['FundReqItemDescription']
  
        
        # ==================================================================
        # need to find the right row of the rel_excel_df 
        # from the relevant excel, find and match the FundReqItemID to the original excel 
        rel_excel_funditem_id = rel_excel_df['FundReqItemID'][rel_excel_df.index[row]]
        print(rel_excel_df['FundReqItemID'][rel_excel_df.index[row]])
                  
        # ==================================================================
        
        
        rel_excel_name = get_excel_name(rel_excel_item_description)
        rel_excel_dates = get_excel_dates(rel_excel_votecode, rel_excel_item_description)

        
        # NOW CHECK
        # make this customisable - let the user choose which methods he/she wants to run 

        result3 = relevant(grant_item_desc, rel_excel_item_description, score)
        
        if check6Bool == 1: 
            result6 = actual_cost(rel_excel_df)
        else:
            result6 = ""
            
            
        if check7Bool == 1: 
            result7 = check_venue(rel_excel_votecode, rel_excel_item_description)
        else: 
            result7 = ""
        
        '''
#         result1 = staff_in_project(rel_excel_name, grant_staff)
#         result2 = check_dates(rel_excel_dates, grant_end_date)
        result3 = relevant(grant_item_desc, rel_excel_item_description, score)
#         result4 = check_traveller(rel_excel_votecode, rel_excel_item_description)
#         result5 = check_votecode(rel_excel_df, rel_excel_votecode)
        result6 = actual_cost(rel_excel_df)
        result7 = check_venue(rel_excel_votecode, rel_excel_item_description)
        '''
        
        resultString = result3 + result6 +  result7
        print(resultString)

        
        # then for each list, append it to the column at the end of the row (column O)
        og_sheet['O'+str(rel_excel_df.index[row] + 1)].value = resultString


        # now highlight
        # if there's only one '|' --> make it yellow 
        # if more than one '|' then highlight red
        # =============== FALSE ===============
    #     redfill = PatternFill(bgColor="FFC7CE", fill_type = "solid")

        if resultString.count('|') > 1 :
#             print("False: " + resultString + " // Highlight: " + "A" + str(rel_excel_df.index[row]))

            for cell in og_sheet[str(rel_excel_df.index[row] + 1) + ":" + str(rel_excel_df.index[row] + 1)]:
                cell.fill = PatternFill("solid", fgColor="FFC7CE")
    #                 og_sheet[str(i+1) + ":" + str(i+1)].fill = PatternFill("solid", fgColor="FFC7CE")


        # =============== MAYBE ===============
        elif resultString.count('|') == 1:
#             print("None: " + resultString + " // Highlight: " + "A" + str(rel_excel_df.index[row]))

            for cell in og_sheet[str(rel_excel_df.index[row] + 1) + ":" + str(rel_excel_df.index[row] + 1)]:
                cell.fill = PatternFill(fill_type="solid", fgColor='FFD642')


            
        # =============== TRUE ===============
        # highlight green
        elif resultString.count('-') >= 1:
            for cell in og_sheet[str(rel_excel_df.index[row] + 1) + ":" + str(rel_excel_df.index[row] + 1)]:
                cell.fill = PatternFill(fill_type="solid", fgColor='B4ED98')
            
        
        # =============== OTHERS (not checked) ===============
        # do nothing
        else:
            for cell in og_sheet[str(rel_excel_df.index[row] + 1) + ":" + str(rel_excel_df.index[row] + 1)]:
                cell.fill = PatternFill(fill_type="solid", fgColor='FFFFFF')

    #             og_sheet[str(i+1) + ":" + str(i+1)].fill = PatternFill(fill_type="solid", fgColor='FFFFFF')
                

        done_filename = '../done_test_fundable_items.xlsx'
        og_wb.save(filename = done_filename)

    # SHOW IN TKINTER - when done 
    tk.Label(window, text = 'Done').grid(row=13)
    

In [7]:
# Upload folder of PDFs
def UploadPDFFolder():
    folder = filedialog.askdirectory(title = "Select a folder containing the grant PDF(s)")
#     directory = os.fsencode(folder)

    print('Selected folder: ', folder)
    tk.Label(window, text = folder).grid(row=4, column=1, padx=(15,0), pady=(15,5), sticky=W)
    return folder



# def UploadPDF():
#     filename = filedialog.askopenfilename(title = "Select the grant PDF")
#     print('Selected pdf: ', filename)
#     tk.Label(window, text = 'Selected pdf:' + filename).grid(row=3)
  
#     if filename:
#         with open(filename) as file:
#             return file.name


        
def UploadExcel():
    filename = filedialog.askopenfilename(title = "Select the excel")
    print('Selected file:', filename)
    tk.Label(window, text = filename).grid(row=5, column=1, padx=(15,0), pady=(0,5), sticky=W)
    
    if filename:
        with open(filename) as file:
            return file.name
    

    
def check(folder, og_wb, score, check6Bool, check7Bool):
    # To loop through pdf forms in uploaded folder 
    for filename in os.listdir(folder):
        print(filename)

        # Get the filename of each file in the folder 
        text = pdf_to_image(folder, filename)

        # Run the functions on each file, one at a time, in a loop 
        highlight(text, og_wb, score, check6Bool, check7Bool)

    

# Splash / Instructions

In [8]:
# =====================================================================
# ============================== SPLASH ===============================
splash = tk.Tk()
splash.title("Welcome to A*GAFS")
splash.grid_columnconfigure(0, weight=0)
splash.geometry("700x500")

# empty row to add space
topSpace = tk.Label(splash, text = "")
topSpace.grid(row=0, column=0)
        
# ========================= IMAGE =================================
image = PIL.Image.open("astarLogoNew.png")
photo = ImageTk.PhotoImage(image)
label = tk.Label(image=photo)
label.image = photo 
label.grid(row=1, column=0, padx=(15,0))


# ========================= TITLE =================================
nameLabel = tk.Label(splash, text = "A*STAR Grant Audit Flagging System")
nameLabel.grid(row=1, column=1, pady=(15,15))
nameLabel.config(font=("Arial", 16))


# ========================= INSTRUCTIONS =================================
insLabel = tk.Label(splash, text = "How to use A*GAFS:")
insLabel.grid(row=3, column=1, pady=(15,15))
insLabel.config(font=("Arial", 12))

insDesc = tk.Label(splash, justify='left', anchor='w', text = "1. Upload the folder containing the grant proposals. \n\n2. Upload the excel of grant purchases. \n\n3. Select the similarity score threshold. \n\n4. Click the 'Check' button. \n\n5. Wait for the program to show 'Done' before opening the new excel.")
insDesc.grid(row=4, column=1, pady=(15,15))
insDesc.config(font=("Arial", 10))


## required to make window show before the program gets to the mainloop
splash.update()


# Start 

In [9]:
# =====================================================================
# =============================== START ===============================

sleep(10)
## finished loading so destroy splash
splash.destroy()

# Create instance
window = tk.Tk()   

window.withdraw()

## show window again
window.deiconify()

# Add a title       
window.title("A*GAFS")

# ===================================================================
# =============================== GUI ===============================
window.geometry("700x500")
window.grid_columnconfigure(0, weight=0)

# empty row to add space
topSpace = tk.Label(window, text = "")
topSpace.grid(row=0, column=0)

# top = tk.Frame(window, width=2000, height= 60)
# top.grid(row=0)
# top.config(background="#F6F6F6")


# ========================= IMAGE =================================
image = PIL.Image.open("astarLogoNew.png")
photo = ImageTk.PhotoImage(image)
label = tk.Label(image=photo)
label.image = photo 
label.grid(row=1, column=0, padx=(15,0))


# ========================= TITLE =================================
nameLabel = tk.Label(window, text = "A*STAR Grant Audit Flagging System")
nameLabel.grid(row=1, column=1, pady=(15,15))
nameLabel.config(font=("Arial", 16))

# tk.Label(window, text = "Upload folder of grant PDF(s):").grid(row=1, column=0)
# tk.Button(window, text='Select grant PDF folder', width=25, command = lambda : UploadPDFFolder()).grid(row=1, column=1)

# tk.Label(window, text = "Upload excel sheet:").grid(row=3, column=0)
# tk.Button(window, text='Select excel file', width=25, command = lambda : UploadExcel()).grid(row=3, column=1)    


# ========================= FILE / FOLDER =========================
tk.Label(window, text='Selected folder: ').grid(row=4, padx=(15,0), pady=(15,5), column=0,sticky=W)
tk.Label(window, text='Selected file: ').grid(row=5, padx=(15,0), pady=(0,5), column=0, sticky=W)
    
folder = UploadPDFFolder()
og_wb = openpyxl.load_workbook(UploadExcel())
    

# ========================= CONFIGURATION =========================
configLabel = tk.Label(window, text = "Configurations")
configLabel.grid(row=6, column=0, sticky=W, padx=(12,0), pady=(15, 0))
configLabel.config(font=("Arial", 14))


# give it a threshold, if the simiarity is above a certain value then it's clear
# if not, then flag
tk.Label(window, text = "Similarity score stringency:").grid(row=7, column=0, pady=(0, 0))
# scoreEntry = tk.Entry(window)
# scoreEntry.grid(row=7, column=0)

tk.Label(window, text = "Not stringent").grid(row=7, column=1, sticky=W, padx=(0, 0), pady=(0, 0))
scoreEntry = Scale(window, from_=0.0, to=1.0, resolution = 0.1, length=240, orient=HORIZONTAL)
scoreEntry.grid(row=7, column=1, padx=(0, 45), pady=(0, 15))
tk.Label(window, text = "Stringent").grid(row=7, column=1, sticky=E, padx=(0, 70), pady=(0,0))

score = scoreEntry.get()

# if the user didn't input a score threshold, the default 0.5 will be used
if (score == ""):
    score = 0.5

    
# Customise the checks by checking the box 
check6Bool = tk.IntVar()
tk.Checkbutton(window, text="Check actual cost", variable=check6Bool).grid(row=9, sticky=W, padx=(15,0))
check7Bool = tk.IntVar()
tk.Checkbutton(window, text="Check venue", variable=check7Bool).grid(row=10, sticky=W, padx=(15,0)) 
    
    
# empty row to add space
tk.Label(window, text = "").grid(row=11, column=0)
# 'command' is executed when you click the button
tk.Button(window, text = "Check", width=20, command = lambda : check(folder, og_wb, score, check6Bool, check7Bool)).grid(row=12, padx=(15,0))

    
window.mainloop()


Selected folder:  /Users/nicoleong/Documents/Anaconda/agafs/Test PDFs
Selected file: /Users/nicoleong/Documents/Anaconda/agafs/test_fundable_items.xlsx
test4.pdf
project id number: 1021660124
| name: bernadette lee

| project title: innovative marine antifouling solutions for high value
applications (imas) - budget for visiting principal scientist prof julius
vancso

project | staff: liu, dean, jerry, ryan, lan, bernadette

| grant date (quarter): 1
| grant date (year): 2014

| end date: 20/1/2016

| item type: exp

| vote code: overseas

| traveller: liu

| account description: overseas travel (airfare/transport)

| item description: a/p liu xg . china 17-21 dec 13 . changsa-tx fee to
explore innovative marine antifouling solutions for high value
applications (imas) - budget for visiting principal scientist prof julius
vancso

| date of application: 20/1/2014

| grant amount: s$ 796.60
1228925

accomodation for g vancso (jan14)
-
1228926

g vancso-reimb airfare(jan14)-eur 3820.15
-
12